In [1]:
import pandas as pd
import numpy as np
import chardet

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd gdrive/MyDrive/News+LPReports/News-Classifiers/summary

/content/gdrive/MyDrive/News+LPReports/News-Classifiers/summary


# Add in short names to find match

In [4]:
df1 = pd.read_csv('data/150621_signal_google_predictions.csv', index_col=0)
df1.reset_index(drop=True, inplace=True)
df1.dropna(subset=['Search Query'], inplace=True)
df1.head(3)

,Search Query,Date,URL,article,revenue,product,market,partnership,mgmt,clinical,fundraising
0,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,It is already impossible for a single entity t...,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,"“I don’t think anyone can shut it down now, gi...",0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,“You can’t erase that.” Fighting off bitcoin a...,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df2 = pd.read_excel(open('data/vpc_master_070621.xlsx', 'rb'), sheet_name='Master')
df2.head(3)

,Vertex Entity,VMF2,Company/Service Name,Company Name for Search,short_names,Optimized Search Engine,UUID,Company Description,Tech Verticals,HQ/Main Operation (Country),Financing Stage,Holding Status,Date of Vertex Investment,Website
0,Vertex Ventures China,NaN,73Hours 高跟73小时,"""高跟73小时""","""高跟73小时""",Sogou,company-https---www-linkedin-com-company-blind...,"Founded in 2015, the company designs and sells...",Consumer Products & Services,China,Series B,Active,2017-01-01,https://73hours.world.taobao.com
1,Vertex Ventures China,NaN,91.com 91无线 (EXITED),NaN,NaN,NaN,NaN,"Founded in 2016, Atidot is a cloud based predi...",Ecommerce,China,Acquired,Exit,2011-12-01,http://www.91.com
2,Vertex Ventures China,NaN,AAG Energy 亚美能源,NaN,NaN,NaN,NaN,AAG Energy Holdings Limited is an internationa...,Oil and Gas; CleanTech,China,IPO,Exit,2012-04-01,http://www.aagenergy.com


In [6]:
# remove companies we are not tracking
df2.dropna(subset=['Company Name for Search'], inplace=True)
df2.head(3)

,Vertex Entity,VMF2,Company/Service Name,Company Name for Search,short_names,Optimized Search Engine,UUID,Company Description,Tech Verticals,HQ/Main Operation (Country),Financing Stage,Holding Status,Date of Vertex Investment,Website
0,Vertex Ventures China,NaN,73Hours 高跟73小时,"""高跟73小时""","""高跟73小时""",Sogou,company-https---www-linkedin-com-company-blind...,"Founded in 2015, the company designs and sells...",Consumer Products & Services,China,Series B,Active,2017-01-01,https://73hours.world.taobao.com
4,Vertex Ventures China,✓,"ArrayComm (""Xiamen ArrayComm Wireless Technolo...",ArrayComm,ArrayComm,Google,company-https---www-linkedin-com-company-array...,ArrayComm is a leading provider of physical la...,Internet of Things (IoT); Communication software,China,Series A,Active,2020-09-11,httphttp://www.arraycomm.com/
6,Vertex Ventures China,✓,BlueX,联睿微电子,联睿微电子,Baidu,company-https---www-linkedin-com-company-blind...,BlueX is developing high-end Bluetooth chips f...,Internet of Things (IoT); Communication Hardwa...,China,Series A,Active,2019-12-02,https://www.bluexmicro.com


In [7]:
df3 = df2[['Company Name for Search', 'short_names']]
df3.head(3)

,Company Name for Search,short_names
0,"""高跟73小时""","""高跟73小时"""
4,ArrayComm,ArrayComm
6,联睿微电子,联睿微电子


In [8]:
# create a dictionary for mapping search query to short names
s = df3.groupby('Company Name for Search')['short_names'].apply(lambda x: x.tolist())
name_dict = s.to_dict()
print(name_dict)

{'"100课堂"': ['"100课堂"'], '"NEIWAI内外"': ['"NEIWAI内外"'], '"乐车邦"': ['"乐车邦"'], '"人健药业"': ['"人健药业"'], '"即刻APP"': ['"即刻APP"'], '"好贷网"': ['"好贷网"'], '"寿光蔬菜产业控股集团"': ['"寿光蔬菜产业控股集团"'], '"晓多料技"': ['"晓多料技"'], '"比科奇"微电子': ['"比科奇"微电子'], '"深视创新"': ['"深视创新"'], '"猛犸反欺诈"': ['"猛犸反欺诈"'], '"积木盒子"': ['"积木盒子"'], '"穿衣助手"': ['"穿衣助手"'], '"网龙华渔"': ['"网龙华渔"'], '"聚土网"': ['"聚土网"'], '"芯驰科技"': ['"芯驰科技"'], '"花卷商城"': ['"花卷商城"'], '"讯创电子"': ['"讯创电子"'], '"钜力能"': ['"钜力能"'], '"高跟73小时"': ['"高跟73小时"'], '17Live': ['17Live'], 'Ace Turtle Services': ['Ace Turtle '], 'Active.ai': ['Active AI'], 'Adaptive Shield': ['Adaptive Shield'], 'Ambidextrous.ai': ['Ambidextrous'], 'Aporia': ['Aporia'], 'ArrayComm': ['ArrayComm'], 'Aruna': ['Aruna'], 'Astound.ai': ['Astound'], 'Atidot': ['Atidot'], 'Axonius': ['Axonius'], 'Bicycle Therapeutics': ['Bicycle '], 'Binance': ['Binance'], 'Bites': ['Bites'], 'BlackThorn Therapeutics': ['BlackThorn'], 'Boundless Bio': ['Boundless Bio'], 'Broker Buddha': ['Broker Buddha'], 'CODA': ['CODA'], 'Cicil s

In [9]:
# map short names to search queries
# remove all whitespaces
df1['short_name'] = df1['Search Query'].apply(lambda x: name_dict[x][0].replace(' ',''))
df1.head(3)

,Search Query,Date,URL,article,revenue,product,market,partnership,mgmt,clinical,fundraising,short_name
0,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,It is already impossible for a single entity t...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Binance
1,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,"“I don’t think anyone can shut it down now, gi...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,Binance
2,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,“You can’t erase that.” Fighting off bitcoin a...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Binance


In [10]:
# match = 1 if short name appears in article
match_list = []
for idx,row in df1.iterrows():
  if row['short_name'] in row['article']:
    match_list.append(1)
  else:
    match_list.append(0)

df1['match'] = match_list
df1.head(3)

,Search Query,Date,URL,article,revenue,product,market,partnership,mgmt,clinical,fundraising,short_name,match
0,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,It is already impossible for a single entity t...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Binance,0
1,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,"“I don’t think anyone can shut it down now, gi...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,Binance,1
2,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,“You can’t erase that.” Fighting off bitcoin a...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Binance,0


## Add in 1 sentence before and after match

In [11]:
# remove indexes if they exceed this range (from original df)
indexes = list(df1.index.values)
first_idx = indexes[0]
last_idx = indexes[-1]

In [12]:
# retrieve indexes with match = 1
df3 = df1[df1['match'] == 1.0]
idx_list = list(df3.index.values)

# add 1 sentence before and 1 sentence after to the list
new_list = []
for i in idx_list:
  prior = i - 1
  new_list.append(prior)

  after = i + 1
  new_list.append(after)

idx_list = idx_list + new_list

# remove duplicates
idx_list = list(dict.fromkeys(idx_list))
# sort in ascending order
idx_list.sort()

# remove first and last element if it exceeds the index range
if idx_list[0] < first_idx:
  idx_list.pop(0)

elif idx_list[-1] > last_idx:
  idx_list.pop(-1)

In [13]:
for idx in idx_list:
  df1.loc[idx, 'match'] = 1

df1.head(3)

,Search Query,Date,URL,article,revenue,product,market,partnership,mgmt,clinical,fundraising,short_name,match
0,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,It is already impossible for a single entity t...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Binance,1.0
1,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,"“I don’t think anyone can shut it down now, gi...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,Binance,1.0
2,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,“You can’t erase that.” Fighting off bitcoin a...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Binance,1.0


# new stuff

In [14]:
# remove sentences without matches
df = df1[df1['match'] == 1]
df.reset_index(inplace=True)
df.head(3)

,index,Search Query,Date,URL,article,revenue,product,market,partnership,mgmt,clinical,fundraising,short_name,match
0,0,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,It is already impossible for a single entity t...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Binance,1.0
1,1,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,"“I don’t think anyone can shut it down now, gi...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,Binance,1.0
2,2,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,“You can’t erase that.” Fighting off bitcoin a...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Binance,1.0


In [15]:
df.shape

(674, 14)

In [16]:
# get dataframe full of signals
signal_df = df.loc[:,'revenue':'fundraising']
signal_df.head(3)

,revenue,product,market,partnership,mgmt,clinical,fundraising
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# remove rows with all signal = 0
signal_df = signal_df.replace(0, np.nan)
signal_df = signal_df.dropna(how='all', axis=0)
signal_df = signal_df.replace(np.nan, 0)
idx_list = list(signal_df.index.values)
print(len(idx_list))

656


In [18]:
# keep only the indexes of rows with at least 1 signal
df = df.iloc[idx_list]
df.head(3)

,index,Search Query,Date,URL,article,revenue,product,market,partnership,mgmt,clinical,fundraising,short_name,match
0,0,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,It is already impossible for a single entity t...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Binance,1.0
1,1,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,"“I don’t think anyone can shut it down now, gi...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,Binance,1.0
2,2,Binance,2021-05-27 22:50:00+00:00,https://www.coindesk.com/binance-cz-consensus-...,“You can’t erase that.” Fighting off bitcoin a...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Binance,1.0


In [19]:
# aggregate data together by a multi-index of search query, followed by index
# this is for the purpose of grouping up continuous texts together later on (e.g. 2731, 2732, 2733)

final_df = df.groupby(['Search Query','index']).agg({
                'article': '.'.join,
                'URL': list
            })
final_df = pd.DataFrame(final_df)
final_df.head(10)

article                                                URL
Search Query index                                                                                                      
17Live       2731   Part of the relief act could be used to pay fo...  [https://www.taipeitimes.com/News/editorials/a...
             2732   On Friday last week, a teacher at Taipei Munic...  [https://www.taipeitimes.com/News/editorials/a...
             2733   While that was creative, it also speaks to a l...  [https://www.taipeitimes.com/News/editorials/a...
             2737   Increased relief funds are a positive developm...  [https://www.taipeitimes.com/News/editorials/a...
Aruna        1545   In a letter addressed to the Nigeria Olympic C...  [https://guardian.ng/sport/ittfs-wild-card-giv...
             1546   Speaking on the new development, NTTF Caretake...  [https://guardian.ng/sport/ittfs-wild-card-giv...
             1547   “When Aruna got injured during the African Qua...  [https://guardian.ng/sport/ittfs-wild-card-giv...
             1548   “Despite the setback in Tunisia, Aruna did not...  [https://guardian.ng/sport/ittfs-wild-card-giv...
             1549   So far, 64 players have been confirmed to take...  [https://guardian.ng/sport/ittfs-wild-card-giv...
             1550   Gaudium Sportopia centre for excellence to be ...  [https://www.thehindu.com/sport/other-sports/g...

In [ ]:
# reset the index so that we obtain the multi-indexes as columns instead
final_df.reset_index(inplace=True)
final_df.head(3)

,Search Query,index,article,URL
0,17Live,2731,Part of the relief act could be used to pay fo...,[https://www.taipeitimes.com/News/editorials/a...
1,17Live,2732,"On Friday last week, a teacher at Taipei Munic...",[https://www.taipeitimes.com/News/editorials/a...
2,17Live,2733,"While that was creative, it also speaks to a l...",[https://www.taipeitimes.com/News/editorials/a...


In [ ]:
final_df.rename(columns={'index':'old_index'}, inplace=True)
final_df.reset_index(inplace=True)
final_df.head(3)

,index,Search Query,old_index,article,URL
0,0,17Live,2731,Part of the relief act could be used to pay fo...,[https://www.taipeitimes.com/News/editorials/a...
1,1,17Live,2732,"On Friday last week, a teacher at Taipei Munic...",[https://www.taipeitimes.com/News/editorials/a...
2,2,17Live,2733,"While that was creative, it also speaks to a l...",[https://www.taipeitimes.com/News/editorials/a...


In [ ]:
# create a new empty column for input of values during iterrows 
final_df['new_index'] = np.nan

# this section is the part where we will use the following logic to join continuous texts together (e.g. 2731, 2732, 2733)
# firstly, if it is the first row, we will just keep the old index as the new index
# if not, we will implement the following logic:
# if the old index of the current row is +1 away from the previous row, we set the new index of current row to be same as previous row
# else, the new index will be set as old index
# in this way, all continuous indexes will be set with the same new index, for grouping together later on
for idx,row in final_df.iterrows():
  if idx == 0:
    final_df.loc[idx, 'new_index'] = row['old_index']

  else:
    previous = final_df.loc[idx-1, 'old_index']
    if row['old_index'] == previous + 1:
      final_df.loc[idx, 'new_index'] = final_df.loc[idx-1, 'new_index']

    else:
      final_df.loc[idx, 'new_index'] = final_df.loc[idx, 'old_index']

final_df.head()

,index,Search Query,old_index,article,URL,new_index
0,0,17Live,2731,Part of the relief act could be used to pay fo...,[https://www.taipeitimes.com/News/editorials/a...,2731.0
1,1,17Live,2732,"On Friday last week, a teacher at Taipei Munic...",[https://www.taipeitimes.com/News/editorials/a...,2731.0
2,2,17Live,2733,"While that was creative, it also speaks to a l...",[https://www.taipeitimes.com/News/editorials/a...,2731.0
3,3,17Live,2737,Increased relief funds are a positive developm...,[https://www.taipeitimes.com/News/editorials/a...,2737.0
4,4,Aruna,1545,In a letter addressed to the Nigeria Olympic C...,[https://guardian.ng/sport/ittfs-wild-card-giv...,1545.0


In [ ]:
df = final_df.groupby(['Search Query','new_index']).agg({
                'article': '.'.join,
                'URL': sum # use sum to join lists together
            })
df = pd.DataFrame(df)
df.head()

article                                                URL
Search Query new_index                                                                                                      
17Live       2731.0     Part of the relief act could be used to pay fo...  [https://www.taipeitimes.com/News/editorials/a...
             2737.0     Increased relief funds are a positive developm...  [https://www.taipeitimes.com/News/editorials/a...
Aruna        1545.0     In a letter addressed to the Nigeria Olympic C...  [https://guardian.ng/sport/ittfs-wild-card-giv...
             1562.0     By Express News Service CHENNAI: The Tamil Nad...  [https://www.newindianexpress.com/states/tamil...
             1577.0     "A maximum of 9 sanitary pads would be given t...  [https://www.5dariyanews.com/news/326724-Aruna...

In [ ]:
df.to_csv('data/180621_grouped.csv')